In [1]:
import datetime
import subprocess
import os

import pandas as pd
from graphreduce.node import DynamicNode
from graphreduce.graph_reduce import GraphReduce
from graphreduce.enum import ComputeLayerEnum, PeriodUnit

/Users/wesmadrigal/projects/graphreduce/venv/lib/python3.9/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
files = [
    'https://raw.githubusercontent.com/wesmadrigal/GraphReduce/master/tests/data/cust_data/cust.csv',
    'https://raw.githubusercontent.com/wesmadrigal/GraphReduce/master/tests/data/cust_data/orders.csv',
]

for f in os.listdir(os.getcwd()):
    for _f in files:
        _fname = _f.split('/')[-1]
        if _fname == f or _fname in f:
            os.remove(f)

for f in files:
  subprocess.run(['wget', f])

--2024-08-30 13:52:35--  https://raw.githubusercontent.com/wesmadrigal/GraphReduce/master/tests/data/cust_data/cust.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35 [text/plain]
Saving to: ‘cust.csv’

     0K                                                       100% 4.77M=0s

2024-08-30 13:52:36 (4.77 MB/s) - ‘cust.csv’ saved [35/35]

--2024-08-30 13:52:36--  https://raw.githubusercontent.com/wesmadrigal/GraphReduce/master/tests/data/cust_data/orders.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191 [text/plain]
Savin

In [3]:
# Need unique prefixes for all nodes
# so when columns are merged we know
# where they originate from.
prefixes = {
    'cust.csv' : {'prefix':'cu'},
    'orders.csv':{'prefix':'ord'}
}

In [14]:
# create graph reduce nodes
gr_nodes = {
    f.split('/')[-1]: DynamicNode(
        fpath=f,
        fmt='csv',
        pk='id',
        prefix=prefixes[f]['prefix'],
        date_key=None,
        compute_layer=ComputeLayerEnum.pandas,
        compute_period_val=730,
        compute_period_unit=PeriodUnit.day,
    )
    for f in prefixes.keys()
}

2024-08-30 14:57:55 [warning  ] no `date_key` set for <GraphReduceNode: fpath=cust.csv fmt=csv>
2024-08-30 14:57:55 [warning  ] no `date_key` set for <GraphReduceNode: fpath=orders.csv fmt=csv>


In [15]:
gr_nodes['cust.csv'].do_data()

In [16]:
type(gr_nodes['cust.csv'].df)

pandas.core.frame.DataFrame

In [17]:
# Use dask now
# create graph reduce nodes
gr_nodes = {
    f.split('/')[-1]: DynamicNode(
        fpath=f,
        fmt='csv',
        pk='id',
        prefix=prefixes[f]['prefix'],
        date_key=None,
        compute_layer=ComputeLayerEnum.dask,
        compute_period_val=730,
        compute_period_unit=PeriodUnit.day,
    )
    for f in prefixes.keys()
}

2024-08-30 14:58:05 [warning  ] no `date_key` set for <GraphReduceNode: fpath=cust.csv fmt=csv>
2024-08-30 14:58:05 [warning  ] no `date_key` set for <GraphReduceNode: fpath=orders.csv fmt=csv>


In [19]:
gr_nodes['cust.csv'].do_data()

In [20]:
type(gr_nodes['cust.csv'].df)

dask.dataframe.core.DataFrame

In [21]:
# This will not run unless you have access to a spark context
